<a href="https://colab.research.google.com/github/skj092/Lab/blob/main/Dog_Breed_identification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the necessary libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf 
from tensorflow.keras import models, layers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
import os, cv2, json 
from PIL import Image 
import keras

In [2]:
# downloading the dataset 


os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle/'

In [3]:
%%bash 

kaggle competitions download -c dog-breed-identification

unzip -q dog-breed-identification.zip 

rm dog-breed-identification.zip

100%|██████████| 691M/691M [00:14<00:00, 48.4MB/s]


In [4]:
# Config 
class CFG:
    WORK_DIR = "/content"
    BATCH_SIZE = 32
    EPOCHS = 5 
    TARGET_SIZE = 224

In [5]:
# model
def create_model():
    conv_base = EfficientNetB0(include_top=False, weights= None, input_shape=(CFG.TARGET_SIZE, CFG.TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(5, activation='softmax')(model)
    model = models.Model(inputs=conv_base.input, outputs=model)
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
import tensorflow as tf 
from tensorflow.keras import layers 

def create_model():
    base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top = False, 
                                                     classes = 120) 
    base_model.trainable = False 

    inputs = layers.Input(shape = (224,224,3))
    x = base_model(inputs, training = False) 
    x = tf.keras.layers.GlobalAveragePooling2D(name= "global_average_pooling")(x)
    x = layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(120, activation="softmax")(x)


    ModelDogBreed = tf.keras.Model(inputs, outputs) 

    ModelDogBreed.compile(loss = "categorical_crossentropy", 
                         optimizer = tf.keras.optimizers.Adam(), 
                         metrics=["accuracy"]) 

    return ModelDogBreed

In [7]:
def activation_layer_vis(img, activation_layer=0, layers=10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(img)
    rows = int(activations[activation_layer].shape[3]/3)
    cols = int(activations[activation_layer].shape[3]/rows)
    fig, ax = plt.subplots(rows, cols, figsize=(15, 15*cols))
    axes = ax.flatten()
    for i, ax in zip(range(activations[activation_layer].shape[3]), axes):
        ax.imshow(activations[activation_layer][0, :, :, i], cmap='viridis')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [8]:
# data
train_labels = pd.read_csv(os.path.join(CFG.WORK_DIR, "labels.csv"))
STEPS_PER_EPOCH = len(train_labels)*0.8//CFG.BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2//CFG.BATCH_SIZE

In [9]:
train_labels['id'] = train_labels['id'] + '.jpg'

In [10]:
train_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever


In [11]:
# data generator
train_labels.label = train_labels.breed.astype(str)
train_datagen = ImageDataGenerator(
    validation_split=0.2, preprocessing_function=None, 
    rotation_range=45, zoom_range=0.2, horizontal_flip=True,
    vertical_flip=True, fill_mode='nearest', shear_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_labels, directory=os.path.join(CFG.WORK_DIR, "train"),
    subset='training', x_col='id', y_col='breed', target_size=(CFG.TARGET_SIZE, CFG.TARGET_SIZE),
    batch_size=CFG.BATCH_SIZE, class_mode="categorical", shuffle=True, seed=42)

validation_dataget = ImageDataGenerator(validation_split=0.2)

validation_generator = validation_dataget.flow_from_dataframe(
    dataframe=train_labels, directory=os.path.join(CFG.WORK_DIR, "train"),
    subset='validation', x_col='id', y_col='breed', target_size=(CFG.TARGET_SIZE, CFG.TARGET_SIZE),
    batch_size=CFG.BATCH_SIZE, class_mode="categorical", shuffle=False, seed=42)

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [12]:
# model 
print('Creating model...')
model = create_model()
model.summary()

Creating model...
9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, None, None, 1280)  2257984  
 ional)                                                          
                                                                 
 global_average_pooling (Glo  (None, 1280)             0         
 balAveragePooling2D)                                            
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 120)               153720    
                                              

In [13]:
# callbacks
model_save = ModelCheckpoint('model.h5', save_best_only=True, save_weight_only=True, monitor='val_loss', mode='min', verbose=1)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, mode='min', restore_best_weights=True, verbose=1)
reduct_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_delta=0.001, mode='min', verbose=1)

In [14]:
# training
print('Training model...')
history = model.fit(
    train_generator, steps_per_epoch=STEPS_PER_EPOCH, epochs=CFG.EPOCHS,
    validation_data=validation_generator, validation_steps=VALIDATION_STEPS,
    callbacks=[model_save, early_stop, reduct_lr])

Training model...
Epoch 1/5
255/255 [==============================] - ETA: 0s - loss: 5.1041 - accuracy: 0.0145
Epoch 1: val_loss improved from inf to 4.85092, saving model to model.h5
255/255 [==============================] - 128s 459ms/step - loss: 5.1041 - accuracy: 0.0145 - val_loss: 4.8509 - val_accuracy: 0.0193 - lr: 0.0010
Epoch 2/5
255/255 [==============================] - ETA: 0s - loss: 4.8401 - accuracy: 0.0255
Epoch 2: val_loss improved from 4.85092 to 4.78632, saving model to model.h5
255/255 [==============================] - 109s 427ms/step - loss: 4.8401 - accuracy: 0.0255 - val_loss: 4.7863 - val_accuracy: 0.0228 - lr: 0.0010
Epoch 3/5
 99/255 [==========>...................] - ETA: 1:02 - loss: 4.7498 - accuracy: 0.0290

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-f5cd8451c2ab>", line 6, in <module>
    callbacks=[model_save, early_stop, reduct_lr])
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1409, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 947, in _call
    return s

KeyboardInterrupt: ignored

In [ ]:
# visualization
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()